In [1]:
# Python code to convert video to audio
import moviepy.editor as mp
import torch
import librosa
import numpy as np
import soundfile as sf
import wave
from scipy.io import wavfile
from IPython.display import Audio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

In [3]:
# Insert Local Video File Path 
clip = mp.VideoFileClip(r"./reuben_deepfake.mp4")
  
# Insert Local Audio File Path
clip.audio.write_audiofile(r"./reuben.wav")

MoviePy - Writing audio in ./reuben.wav


MoviePy - Done.


In [3]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
import wave
import json

from vosk import Model, KaldiRecognizer, SetLogLevel
import Word as custom_Word

model_path = "vosk-model-en-us-0.21"
audio_filename = "transcript.wav"

model = Model(model_path)
wf = wave.open(audio_filename, "rb")
rec = KaldiRecognizer(model, wf.getframerate())
rec.SetWords(True)

# get the list of JSON dictionaries
results = []
# recognize speech using vosk model
while True:
    data = wf.readframes(4000)
    if len(data) == 0:
        break
    if rec.AcceptWaveform(data):
        part_result = json.loads(rec.Result())
        results.append(part_result)
part_result = json.loads(rec.FinalResult())
results.append(part_result)

# convert list of JSON dictionaries to list of 'Word' objects
list_of_Words = []
for sentence in results:
    if len(sentence) == 1:
        # sometimes there are bugs in recognition 
        # and it returns an empty dictionary
        # {'text': ''}
        continue
    for obj in sentence['result']:
        w = custom_Word.Word(obj)  # create custom Word object
        list_of_Words.append(w)  # and add it to list

wf.close()  # close audiofile

# output to the screen
for word in list_of_Words:
    print(word.to_string())

more                 from 1.41 sec to 1.83 sec, confidence is 66.42%
and                  from 1.83 sec to 2.19 sec, confidence is 100.00%
torn                 from 2.19 sec to 2.76 sec, confidence is 100.00%
an                   from 2.76 sec to 3.03 sec, confidence is 79.73%
hour                 from 3.06 sec to 4.05 sec, confidence is 79.73%
the                  from 4.23 sec to 4.41 sec, confidence is 100.00%
whole                from 4.41 sec to 4.95 sec, confidence is 100.00%
armor                from 4.95 sec to 5.34 sec, confidence is 60.49%
mashed               from 5.34 sec to 6.00 sec, confidence is 91.17%
but                  from 6.00 sec to 6.55 sec, confidence is 83.27%
look                 from 6.60 sec to 7.02 sec, confidence is 52.81%
where                from 7.02 sec to 7.38 sec, confidence is 86.14%
are                  from 7.56 sec to 8.01 sec, confidence is 89.74%
you                  from 8.01 sec to 8.29 sec, confidence is 89.74%
i'm                  from 8.29

In [4]:
import speech_recognition as sr

r = sr.Recognizer()

with sr.AudioFile('transcript.wav') as source:
    audio = r.record(source, duration=30)
    text = ""
    try:
        while True:
            text += r.recognize_google(audio)
            audio = r.record(source, duration=30)
    except sr.WaitTimeoutError:
        pass

with open('output.txt', 'w') as f:
    f.write(text)


result2:
{   'alternative': [   {   'confidence': 0.8600024,
                           'transcript': 'entering an error which are Enemies '
                                         'committed look like anyone is saying '
                                         'anything have me say things like '
                                         'now'},
                       {   'transcript': 'which are Enemies committed look '
                                         'like anyone is saying anything have '
                                         'me say things like now'},
                       {   'transcript': 'entering an error which are Enemies '
                                         'committed look like anyone saying '
                                         'anything have me say things like '
                                         'now'},
                       {   'transcript': 'entering an error which are Enemies '
                                         'committed look like a

UnknownValueError: 

In [5]:
file_name = 'transcript.wav'

data = wavfile.read(file_name)
framerate = data[0]
sounddata = data[1]
time = np.arange(0,len(sounddata))/framerate
input_audio, _ = librosa.load(file_name, sr=16000)
input_values = tokenizer(input_audio, return_tensors="pt").input_values
logits = model(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)
transcription = tokenizer.batch_decode(predicted_ids)[0]
print(transcription)


WE'RE ENTERING AN ERA IN WHICH OUR ENEMIES CAN MAKE IT LOOK LIKE ANY ONE IS SAYING ANYTHING AT ANY POINT IN TIME EVEN IF THEY WOULD NEVER SAY THOSE THINGS SO FOR INSTANCE THEY COULD HAVE ME SAY THINGS LIKE I'LL KNOW KILMONGER WAS RIGHT OR BEN CARSON IS IN THE SUNKIN PLACES OR ABOUT THIS SIMPLY PRESIDENT TRUMP IS A TOTAL AND COMPLETE DIPSHIT NOW YOU SEE I WOULD NEVER SAY THESE THINGS AT LEAST NOT IN A PUBLIC ADDRESS BUT SOME ONE ELSE WOULD SOME ONE LIKE GEORGAN PEAL THIS IS A DANGEROUS TIME MOVING FORWARD WE NEED TO BE MORE VIGILANT WITH WHAT WE TRUST FROM THE INNER NET THAT'S A TIME WHEN WE NEED TO RELIVE ON TRUSTED NEW SOURCES MAY SOUND BASIC BUT HOW WE MOVE FORWARD IN THE AGE OF INFORMATION IS GOING TO BE THE DIFFERENCE BETWEEN WHETHER WE SURVIVE OR WHETHER WE BECOME SOME KIND OF FUCKED UP DISTOPIA THANK YOU STAY WOKE BITCHES


In [17]:
import speech_recognition as sr

# Load the audio file
r = sr.Recognizer()
audio_file = sr.AudioFile('transcript.wav')
with audio_file as source:
    audio_data = r.record(source)

# Perform speech recognition
transcription = r.recognize_google(audio_data)

print(transcription)

# Split the transcription into words
words = transcription.split()

# Get the sample rate and number of samples from the audio data
sample_rate = audio_data.sample_rate
num_samples = len(audio_data.get_raw_data())

# Calculate the duration of the audio data
duration = num_samples / sample_rate

print(duration)

# Find the time stamps of the phonemes 'b', 'm', and 'p'
for i, word in enumerate(words):
    if 'b' in word or 'm' in word or 'p' in word:
        word_duration = duration / len(words)
        time_stamp = i * word_duration
        minutes, seconds = divmod(time_stamp, 60)
        milliseconds = int((seconds % 1) * 1000)
        print(f"Phoneme 'b', 'm', or 'p' found in word '{word}' at {int(minutes)}:{int(seconds)}.{milliseconds}")


result2:
{   'alternative': [   {   'confidence': 0.85035503,
                           'transcript': 'entering an error which are Enemies '
                                         'committed look like anyone is saying '
                                         'anything'},
                       {   'transcript': 'which are Enemies committed look '
                                         'like anyone is saying anything'},
                       {   'transcript': 'entering an error which are Enemies '
                                         'committed look like anyone is saying '
                                         'anything have me say things like '
                                         'now'}],
    'final': True}
entering an error which are Enemies committed look like anyone is saying anything
144.94
Phoneme 'b', 'm', or 'p' found in word 'Enemies' at 0:55.746
Phoneme 'b', 'm', or 'p' found in word 'committed' at 1:6.895


In [24]:
import librosa
import numpy as np
import torch
import wave
import contextlib

filename = 'transcript.wav'

with contextlib.closing(wave.open(filename,'r')) as f:
    rate, audio_data = wavfile.read(filename)
    audio_data = audio_data.T
    audio_data = audio_data / np.max(np.abs(audio_data))  # normalize the audio data to the range [-1, 1]

    mfccs = librosa.feature.mfcc(y=audio_data, sr=rate, n_mfcc=13)

    model = torch.load("acoustic_model.pt")  # load a pre-trained acoustic model
    inputs = torch.tensor(mfccs.T).unsqueeze(0)  # add a batch dimension to the inputs
    outputs = model(inputs)  # run the inputs through the acoustic model
    _, predicted_phonemes = torch.max(outputs, dim=2)  # get the predicted phonemes

    for i, predicted_phoneme in enumerate(predicted_phonemes[0]):
        if predicted_phoneme == "M":  # check if the predicted phoneme is "M"
            time = i * (len(audio_data) / mfccs.shape[1]) / rate
            print("Phonetic sound of M found at time: {:.2f} seconds".format(time))


FileNotFoundError: [Errno 2] No such file or directory: 'acoustic_model.pt'

In [18]:
import speech_recognition as sr

# Load the audio file into memory
r = sr.Recognizer()
with sr.AudioFile('transcript.wav') as source:
    audio = r.record(source)

# Recognize the speech in the audio file
text = r.recognize_google(audio)

# Split the text into words
words = text.split()

# Keep track of the time stamp for each word
time_stamps = []
start_time = 0
for word in words:
    end_time = start_time + len(word) / 11025 # Assuming a sample rate of 11025 Hz
    time_stamps.append((start_time, end_time))
    start_time = end_time

# Look for occurrences of "m", "b", and "p"
letter_time_stamps = {}
for letter in "mbp":
    letter_time_stamps[letter] = []

for i, word in enumerate(words):
    for j, char in enumerate(word):
        if char.lower() in "mbp":
            letter_time_stamps[char.lower()].append(
                (time_stamps[i][0] + j / len(word), time_stamps[i][0] + (j + 1) / len(word))
            )

# Print the time stamps for each letter
for letter, stamps in letter_time_stamps.items():
    print(f"{letter}:")
    for start, end in stamps:
        print(f"\t{start:.2f} - {end:.2f}")

result2:
{   'alternative': [   {   'confidence': 0.85986006,
                           'transcript': 'entering an error which are Enemies '
                                         'committed look like anyone is saying '
                                         'anything have me say things like '
                                         'now'},
                       {   'transcript': 'which are Enemies committed look '
                                         'like anyone is saying anything have '
                                         'me say things like now'},
                       {   'transcript': 'entering an error which are Enemies '
                                         'committed look like anyone saying '
                                         'anything have me say things like '
                                         'now'},
                       {   'transcript': 'entering an error which are Enemies '
                                         'committed look like 